## 检查模型和给定的权重是否匹配

### Resnet-MTB

#### 加载模型

In [13]:
## 输入模型
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



# parameters
## input_channels = [256,512,1024,2048]
## attention_channels = 2048



class MTB(nn.Module):
    def __init__(self,in_channel,input_channels,attention_channels,outchannels):
        super(MTB, self).__init__()
        self.input_channels = input_channels
        ## 定义多个channels，得到多尺度特征【Batch，256，1】，【Batch，512，1】，【Batch，1024，1】，【Batch，2048，1】
        self.conv1 =  nn.ModuleList()
        for i in input_channels:
            temp_part = nn.Sequential(

                nn.Conv1d(in_channels=in_channel, out_channels=i, kernel_size=1),
                nn.BatchNorm1d(i),
                nn.ReLU(inplace=True)

            )
            self.conv1.append(temp_part)

        self.conv2 = nn.ModuleList()
        for i in input_channels:
            temp_part_2 = nn.Sequential(
                nn.Conv1d(in_channels=i, out_channels=attention_channels, kernel_size=1),
                nn.BatchNorm1d(attention_channels),
                nn.ReLU(inplace=True)

            )
            self.conv2.append(temp_part_2)
        self.conv4 = nn.ModuleList()
        for i in input_channels:
            temp_part_4 = nn.Sequential(
                nn.Conv1d(in_channels=attention_channels, out_channels=outchannels, kernel_size=1),
                nn.BatchNorm1d(outchannels),
                nn.ReLU(inplace=True)

            )
            self.conv4.append(temp_part_4)

        self.reg = nn.Sequential(
            nn.Linear(in_features=outchannels * len(self.input_channels) * 30, out_features= 2048 ),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),
            
            
    
            
            nn.Linear(2048,1)
     
            
        )


    def  forward(self,x):
        outs = [in_channel(x) for in_channel in self.conv1]
        outs = [in_channel(outs[idx])for idx,in_channel in enumerate(self.conv2)]

        outs = [in_channel(outs[idx]) for idx, in_channel in enumerate(self.conv4)]

        input_feature = torch.cat(outs,dim = 1 )
        input_feature = input_feature.view(input_feature.shape[0],-1)
        outs = self.reg(input_feature)


        return outs


####　测试部分
## 现在我们得到了多个尺度的特征



# feature = torch.rand([2,2048,30])
# in_channel = 2048
# input_channels = [256,512,1024,2048]
# attention_channels = 2048
# outchannels = 1024
# model = MTB(in_channel = in_channel, input_channels=input_channels,attention_channels= attention_channels,outchannels=outchannels)
# print(model)
# 
# results = model(feature)
# #
# print(results.shape)


in_channel = 2048
input_channels = [256, 512, 1024, 2048]
attention_channels = 2048
outchannels = 512
model = MTB(in_channel=in_channel, input_channels=input_channels, attention_channels=attention_channels,
            outchannels=outchannels)


#### 读取权重

In [14]:
model.load_state_dict(torch.load("/hy-tmp/Resnet_pth/Epcoh_46_Rmse_6.845022201538086_PCC_0.2333487301766989_CCC_0.13534336370107866.pth"))  #model.load_state_dict()函数把加载的权重复制到模型的权重中去

<All keys matched successfully>

### Resnet-MTA

#### 加载模型

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



# parameters
## input_channels = [256,512,1024,2048]
## attention_channels = 2048

class NonLocalBlock(nn.Module):
    """ NonLocalBlock Module"""

    def __init__(self, in_channels):
        super(NonLocalBlock, self).__init__()

        conv_nd = nn.Conv1d

        self.in_channels = in_channels
        self.inter_channels = self.in_channels // 2

        self.ImageAfterASPP_bnRelu = nn.Sequential(
            nn.BatchNorm1d(self.in_channels),
            nn.ReLU(inplace=True),
        )

        self.DepthAfterASPP_bnRelu = nn.Sequential(
            nn.BatchNorm1d(self.in_channels),
            nn.ReLU(inplace=True),
        )

        self.R_g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)
        self.R_theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                               kernel_size=1, stride=1, padding=0)
        self.R_phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.R_W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                           kernel_size=1, stride=1, padding=0)

        self.F_g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)
        self.F_theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                               kernel_size=1, stride=1, padding=0)
        self.F_phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.F_W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                           kernel_size=1, stride=1, padding=0)

    def forward(self, self_fea, mutual_fea, alpha, selfImage):

        if selfImage:

            selfNonLocal_fea = self.ImageAfterASPP_bnRelu(self_fea)
            mutualNonLocal_fea = self.DepthAfterASPP_bnRelu(mutual_fea)

            batch_size = selfNonLocal_fea.size(0)
            g_x = self.R_g(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            g_x = g_x.permute(0, 2, 1)
            # using mutual feature to generate attention
            theta_x = self.F_theta(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            phi_x = self.F_phi(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            f = torch.matmul(theta_x, phi_x)

            # using self feature to generate attention
            self_theta_x = self.R_theta(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_theta_x = self_theta_x.permute(0, 2, 1)
            self_phi_x = self.R_phi(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_f = torch.matmul(self_theta_x, self_phi_x)
            # add self_f and mutual f
            f_div_C = F.softmax(alpha * f + self_f, dim=-1)
            y = torch.matmul(f_div_C, g_x)
            y = y.permute(0, 2, 1).contiguous()
            y = y.view(batch_size, self.inter_channels, *selfNonLocal_fea.size()[2:])
            W_y = self.R_W(y)
            z = W_y + self_fea
            return z

        else:
            selfNonLocal_fea = self.DepthAfterASPP_bnRelu(self_fea)## [30,2408,1]


            mutualNonLocal_fea = self.ImageAfterASPP_bnRelu(mutual_fea)##[30,2048,1]

            batch_size = selfNonLocal_fea.size(0) ##30

            g_x = self.F_g(selfNonLocal_fea).view(batch_size, self.inter_channels, -1) ##[30,1,1024]
            g_x = g_x.permute(0, 2, 1)

            # using mutual feature to generate attention
            theta_x = self.R_theta(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            phi_x = self.R_phi(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            f = torch.matmul(theta_x, phi_x)

            # using self feature to generate attention
            self_theta_x = self.F_theta(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_theta_x = self_theta_x.permute(0, 2, 1)
            self_phi_x = self.F_phi(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_f = torch.matmul(self_theta_x, self_phi_x)

            # add self_f and mutual f
            f_div_C = F.softmax(alpha * f + self_f, dim=-1)
            print(g_x.shape)
            print(f_div_C.shape)
            y = torch.matmul(f_div_C, g_x)
            y = y.permute(0, 2, 1).contiguous()
            y = y.view(batch_size, self.inter_channels, *selfNonLocal_fea.size()[2:])
            W_y = self.F_W(y)
            z = W_y + self_fea
            return z

class MTA(nn.Module):
    def __init__(self,in_channel,input_channels,attention_channels,outchannels):
        super(MTA, self).__init__()
        self.input_channels = input_channels
        ## 定义多个channels，得到多尺度特征【Batch，256，1】，【Batch，512，1】，【Batch，1024，1】，【Batch，2048，1】
        self.conv1 =  nn.ModuleList()
        for i in input_channels:
            temp_part = nn.Sequential(

                nn.Conv1d(in_channels=in_channel, out_channels=i, kernel_size=1),
                nn.BatchNorm1d(i),
                nn.ReLU(inplace=True)

            )
            self.conv1.append(temp_part)

        self.conv2 = nn.ModuleList()
        for i in input_channels:
            temp_part_2 = nn.Sequential(
                nn.Conv1d(in_channels=i, out_channels=attention_channels, kernel_size=1),
                nn.BatchNorm1d(attention_channels),
                nn.ReLU(inplace=True)

            )
            self.conv2.append(temp_part_2)
        ## 通过attetnion 需要将他们对其到同一个尺度 return list[Batch,2048,1] * 4
        self.conv3 = nn.Conv1d(in_channels= attention_channels *2,out_channels=2, kernel_size=1)
        self.nonblock = NonLocalBlock(in_channels= attention_channels)
        self.conv4 = nn.ModuleList()
        for i in input_channels:
            temp_part_4 = nn.Sequential(
                nn.Conv1d(in_channels=attention_channels, out_channels=outchannels, kernel_size=1),
                nn.BatchNorm1d(outchannels),
                nn.ReLU(inplace=True)

            )
            self.conv4.append(temp_part_4)

        self.reg = nn.Sequential(
            nn.Linear(in_features=outchannels * len(self.input_channels) * 30, out_features= 2048 ),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),
            nn.Linear(2048,1)
        )

        # self.conv4 = nn.ModuleList([nn.Conv1d(in_channels= attention_channels,out_channels=outchannels,kernel_size=1) for i in range(len(input_channels))])

    def  forward(self,x):
        outs = [in_channel(x) for in_channel in self.conv1]
        outs = [in_channel(outs[idx])for idx,in_channel in enumerate(self.conv2)]

        if len(self.input_channels) == 4:
            conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
            conncat_tensor_01_conv = self.conv3(conncat_tensor_01)
            alpha_01 = F.softmax(conncat_tensor_01_conv,dim=1)
            alpha_01_1 = alpha_01[:,1,:].unsqueeze(dim=2)
            feature_attention_0 = self.nonblock(outs[0], outs[1], alpha_01_1, True)

            conncat_tensor_12 = torch.cat([outs[1], outs[2]], dim=1)
            conncat_tensor_12_conv = self.conv3(conncat_tensor_12)
            alpha_12 = F.softmax(conncat_tensor_12_conv, dim=1)
            alpha_12_2 = alpha_12[:, 1, :].unsqueeze(dim=2)
            feature_attention_1 = self.nonblock(outs[1], outs[2], alpha_12_2, True)

            conncat_tensor_23 = torch.cat([outs[2], outs[3]], dim=1)
            conncat_tensor_23_conv = self.conv3(conncat_tensor_23)
            alpha_23 = F.softmax(conncat_tensor_23_conv, dim=1)
            alpha_23_3 = alpha_23[:, 1, :].unsqueeze(dim=2)
            feature_attention_2 = self.nonblock(outs[2], outs[3], alpha_23_3, True)

            conncat_tensor_30 = torch.cat([outs[3], outs[0]], dim=1)
            conncat_tensor_30_conv = self.conv3(conncat_tensor_30)
            alpha_30 = F.softmax(conncat_tensor_30_conv, dim=1)
            alpha_30_3 = alpha_30[:, 1, :].unsqueeze(dim=2)
            feature_attention_3 = self.nonblock(outs[3], outs[0], alpha_30_3, True)

            outs = [feature_attention_0, feature_attention_1, feature_attention_2, feature_attention_3]
        elif len(self.input_channels) == 3:
            conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
            conncat_tensor_01_conv = self.conv3(conncat_tensor_01)
            alpha_01 = F.softmax(conncat_tensor_01_conv, dim=1)
            alpha_01_1 = alpha_01[:, 1, :].unsqueeze(dim=2)
            feature_attention_0 = self.nonblock(outs[0], outs[1], alpha_01_1, True)

            conncat_tensor_12 = torch.cat([outs[1], outs[2]], dim=1)
            conncat_tensor_12_conv = self.conv3(conncat_tensor_12)
            alpha_12 = F.softmax(conncat_tensor_12_conv, dim=1)
            alpha_12_2 = alpha_12[:, 1, :].unsqueeze(dim=2)
            feature_attention_1 = self.nonblock(outs[1], outs[2], alpha_12_2, True)

            conncat_tensor_20 = torch.cat([outs[2], outs[0]], dim=1)
            conncat_tensor_20_conv = self.conv2(conncat_tensor_20)
            alpha_20 = F.softmax(conncat_tensor_20_conv, dim=1)
            alpha_20_2 = alpha_20[:, 1, :].unsqueeze(dim=2)
            feature_attention_2 = self.nonblock(outs[2], outs[0], alpha_20_2, True)

            outs = [feature_attention_0, feature_attention_1, feature_attention_2]

        outs = [in_channel(outs[idx]) for idx, in_channel in enumerate(self.conv4)]

        input_feature = torch.cat(outs,dim = 1 )
        input_feature = input_feature.view(input_feature.shape[0],-1)
        outs = self.reg(input_feature)


        return outs


####　测试部分
## 现在我们得到了多个尺度的特征
#
# x1 = torch.rand([30,2048,1])
# x2 = torch.rand([30,2048,1])
# x3 = torch.rand([30,2048,1])
# x4 = torch.rand([30,2048,1])
# outs = [x1,x2,x3,x4]
# ### 1. concat feature
#
# conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
# print(conncat_tensor_01.shape)
# conv1 = nn.Conv1d(in_channels=2048 *2 , out_channels=2, kernel_size=1)
# conncat_tensor_01_conv = conv1(conncat_tensor_01)
# alpha_01 = F.softmax(conncat_tensor_01_conv,dim=1)
# alpha_0 = alpha_01[:,0,:]
# alpha_1 = alpha_01[:,1,:].unsqueeze(dim=2)






# nonblock = NonLocalBlock(in_channels= 2048)
# temp_feature_0 = nonblock(outs[0], outs[1], alpha_1, False)



# conv1 = nn.Conv1d(in_channels=2048 * 2 ,out_channels=2,kernel_size=1)
# print(conv1(conncat_tensor_01).shape)

# feature = torch.rand([2,2048,30])
# in_channel = 2048
# input_channels = [256,512,1024,2048]
# attention_channels = 2048
# outchannels = 1024
# model = MTA(in_channel = in_channel, input_channels=input_channels,attention_channels= attention_channels,outchannels=outchannels)
# print(model)
#
# results = model(feature)
#
# print(results.shape)


in_channel = 2048
input_channels = [256, 512, 1024, 2048]
attention_channels = 2048
outchannels = 512
model = MTA(in_channel=in_channel, input_channels=input_channels, attention_channels=attention_channels,
            outchannels=outchannels)



#### 读取权重

In [11]:
model.load_state_dict(torch.load("/hy-tmp/Resnet_pth/Epcoh_159_Rmse_6.803299427032471_PCC_0.2584606433517226_CCC_0.15121600196380586.pth"))  #model.load_state_dict()函数把加载的权重复制到模型的权重中去

<All keys matched successfully>

### Resnet-NS

#### 加载模型

In [17]:
import torch
import torch.nn as nn
# 计算无关特征网络
class unrelated_conv(nn.Module):
    def __init__(self):
        super(unrelated_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 2048, out_channels= 512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512,out_channels=128,kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)

    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        rough_result= self.maxpool_2(temp2_relu)

        rough_result = self.conv3(rough_result)
        rough_result = self.relu_3(rough_result)
        rough_result = self.maxpool_3(rough_result)




        return rough_result

# 计算精细特征网络
class related_conv(nn.Module):
    def __init__(self):
        super(related_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2048, out_channels=512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512, out_channels=128, kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)



    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        smooth_result = self.maxpool_2(temp2_relu)

        smooth_result = self.conv3(smooth_result)
        smooth_result = self.relu_3(smooth_result)
        smooth_result = self.maxpool_3(smooth_result)


        return smooth_result
#输入数据为精细特征，预测结果
class predict_part(nn.Module):
    def __init__(self):
        super(predict_part, self).__init__()
        self.fc1 = nn.Linear(in_features=32,out_features=1)
        self.relu = nn.ReLU(True)
        self.fc2 = nn.Linear(in_features=64,out_features=1)
    def forward(self,x):
        x_in = x.reshape(x.shape[0],-1)
        x_in = self.relu(x_in)
        predict_temp1 = self.fc1(x_in)
        # predict_temp2 = self.relu(predict_temp1)
        # predict_result = self.relu(self.fc2(predict_temp2))

        return predict_temp1
#特征复原。。
class encoder_image(nn.Module):
    def __init__(self):
        super(encoder_image, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=128, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=512, kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=512, out_channels=2048, kernel_size=1)

    def forward(self, smooth_feature, rough_feature):
        encode_feature = torch.add(smooth_feature, rough_feature)
        encode_1 = self.conv1(encode_feature)
        x_hat = self.conv2(encode_1)

        x_hat = self.conv3(x_hat)
        return x_hat



class DSN(nn.Module):
    def __init__(self):
        super(DSN, self).__init__()
        self.unrelated_conv = unrelated_conv()
        self.related_conv = related_conv()
        self.predict_part = predict_part()
        self.encoder = encoder_image()

    def forward(self,X_feature):

        unrealted_data = self.unrelated_conv(X_feature)


        realted_data = self.related_conv(X_feature)


        predict_result  = self.predict_part(realted_data)


        encode_result = self.encoder(unrealted_data, realted_data)

        return unrealted_data, realted_data, predict_result, encode_result

# model = DSN()
# x = torch.rand([600,1,2048])
# x = x.permute(0,2,1)
# print(f'x:{x.shape}')
# unrealted_data, realted_data, predict_result, encode_result = model(x)
# print(f'unrealted_data:{unrealted_data.shape}')
# print(f'predict_Result :{predict_result.shape}')
# print(f'related_data:{realted_data.shape}')
# print(f'encode_result:{encode_result.shape}')
model = DSN()

#### 读取权重

In [18]:
model.load_state_dict(torch.load("/hy-tmp/Resnet_pth/Epcoh_1049_Rmse_6.62206506729126_PCC_0.2721236440806091_CCC_0.16563803434174354.pth"))  #model.load_state_dict()函数把加载的权重复制到模型的权重中去

<All keys matched successfully>

### AU-MTB

#### 加载模型

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



# parameters
## input_channels = [256,512,1024,2048]
## attention_channels = 2048



class MTB(nn.Module):
    def __init__(self,in_channel,input_channels,attention_channels,outchannels):
        super(MTB, self).__init__()
        self.input_channels = input_channels
        ## 定义多个channels，得到多尺度特征【Batch，256，1】，【Batch，512，1】，【Batch，1024，1】，【Batch，2048，1】
        self.conv1 =  nn.ModuleList()
        for i in input_channels:
            temp_part = nn.Sequential(

                nn.Conv1d(in_channels=in_channel, out_channels=i, kernel_size=1),
                nn.BatchNorm1d(i),
                nn.ReLU(inplace=True)

            )
            self.conv1.append(temp_part)

        self.conv2 = nn.ModuleList()
        for i in input_channels:
            temp_part_2 = nn.Sequential(
                nn.Conv1d(in_channels=i, out_channels=attention_channels, kernel_size=1),
                nn.BatchNorm1d(attention_channels),
                nn.ReLU(inplace=True)

            )
            self.conv2.append(temp_part_2)
        self.conv4 = nn.ModuleList()
        for i in input_channels:
            temp_part_4 = nn.Sequential(
                nn.Conv1d(in_channels=attention_channels, out_channels=outchannels, kernel_size=1),
                nn.BatchNorm1d(outchannels),
                nn.ReLU(inplace=True)

            )
            self.conv4.append(temp_part_4)

        self.reg = nn.Sequential(
            nn.Linear(in_features=outchannels * len(self.input_channels) * 30, out_features= 4096 ),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),
            
            
            nn.Linear(in_features=4096 , out_features= 512 ),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),

            
            nn.Linear(512,1)
     
            
        )


    def  forward(self,x):
        outs = [in_channel(x) for in_channel in self.conv1]
        outs = [in_channel(outs[idx])for idx,in_channel in enumerate(self.conv2)]

        outs = [in_channel(outs[idx]) for idx, in_channel in enumerate(self.conv4)]

        input_feature = torch.cat(outs,dim = 1 )
        input_feature = input_feature.view(input_feature.shape[0],-1)
        outs = self.reg(input_feature)


        return outs


####　测试部分
## 现在我们得到了多个尺度的特征



# feature = torch.rand([2,2048,30])
# in_channel = 2048
# input_channels = [256,512,1024,2048]
# attention_channels = 2048
# outchannels = 1024
# model = MTB(in_channel = in_channel, input_channels=input_channels,attention_channels= attention_channels,outchannels=outchannels)
# print(model)
# 
# results = model(feature)
# #
# print(results.shape)

### 
in_channel = 31
input_channels = [4, 8, 16, 31]
attention_channels = 16
outchannels = 16
model = MTB(in_channel=in_channel, input_channels=input_channels, attention_channels=attention_channels,
            outchannels=outchannels)




#### 读取权重

In [35]:
model.load_state_dict(torch.load('/hy-tmp/AU_pth/Epcoh_90_Rmse_6.157999515533447_PCC_0.3665695023234526_CCC_0.24920727533836307.pth'))

<All keys matched successfully>

### AU-MTA

#### 加载模型

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



# parameters
## input_channels = [256,512,1024,2048]
## attention_channels = 2048

class NonLocalBlock(nn.Module):
    """ NonLocalBlock Module"""

    def __init__(self, in_channels):
        super(NonLocalBlock, self).__init__()

        conv_nd = nn.Conv1d

        self.in_channels = in_channels
        self.inter_channels = self.in_channels // 2

        self.ImageAfterASPP_bnRelu = nn.Sequential(
            nn.BatchNorm1d(self.in_channels),
            nn.ReLU(inplace=True),
        )

        self.DepthAfterASPP_bnRelu = nn.Sequential(
            nn.BatchNorm1d(self.in_channels),
            nn.ReLU(inplace=True),
        )

        self.R_g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)
        self.R_theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                               kernel_size=1, stride=1, padding=0)
        self.R_phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.R_W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                           kernel_size=1, stride=1, padding=0)

        self.F_g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)
        self.F_theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                               kernel_size=1, stride=1, padding=0)
        self.F_phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.F_W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                           kernel_size=1, stride=1, padding=0)

    def forward(self, self_fea, mutual_fea, alpha, selfImage):

        if selfImage:

            selfNonLocal_fea = self.ImageAfterASPP_bnRelu(self_fea)
            mutualNonLocal_fea = self.DepthAfterASPP_bnRelu(mutual_fea)

            batch_size = selfNonLocal_fea.size(0)
            g_x = self.R_g(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            g_x = g_x.permute(0, 2, 1)
            # using mutual feature to generate attention
            theta_x = self.F_theta(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            phi_x = self.F_phi(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            f = torch.matmul(theta_x, phi_x)

            # using self feature to generate attention
            self_theta_x = self.R_theta(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_theta_x = self_theta_x.permute(0, 2, 1)
            self_phi_x = self.R_phi(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_f = torch.matmul(self_theta_x, self_phi_x)
            # add self_f and mutual f
            f_div_C = F.softmax(alpha * f + self_f, dim=-1)
            y = torch.matmul(f_div_C, g_x)
            y = y.permute(0, 2, 1).contiguous()
            y = y.view(batch_size, self.inter_channels, *selfNonLocal_fea.size()[2:])
            W_y = self.R_W(y)
            z = W_y + self_fea
            return z

        else:
            selfNonLocal_fea = self.DepthAfterASPP_bnRelu(self_fea)## [30,2408,1]


            mutualNonLocal_fea = self.ImageAfterASPP_bnRelu(mutual_fea)##[30,2048,1]

            batch_size = selfNonLocal_fea.size(0) ##30

            g_x = self.F_g(selfNonLocal_fea).view(batch_size, self.inter_channels, -1) ##[30,1,1024]
            g_x = g_x.permute(0, 2, 1)

            # using mutual feature to generate attention
            theta_x = self.R_theta(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            theta_x = theta_x.permute(0, 2, 1)
            phi_x = self.R_phi(mutualNonLocal_fea).view(batch_size, self.inter_channels, -1)
            f = torch.matmul(theta_x, phi_x)

            # using self feature to generate attention
            self_theta_x = self.F_theta(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_theta_x = self_theta_x.permute(0, 2, 1)
            self_phi_x = self.F_phi(selfNonLocal_fea).view(batch_size, self.inter_channels, -1)
            self_f = torch.matmul(self_theta_x, self_phi_x)

            # add self_f and mutual f
            f_div_C = F.softmax(alpha * f + self_f, dim=-1)
            print(g_x.shape)
            print(f_div_C.shape)
            y = torch.matmul(f_div_C, g_x)
            y = y.permute(0, 2, 1).contiguous()
            y = y.view(batch_size, self.inter_channels, *selfNonLocal_fea.size()[2:])
            W_y = self.F_W(y)
            z = W_y + self_fea
            return z

class MTA(nn.Module):
    def __init__(self,in_channel,input_channels,attention_channels,outchannels):
        super(MTA, self).__init__()
        self.input_channels = input_channels
        ## 定义多个channels，得到多尺度特征【Batch，256，1】，【Batch，512，1】，【Batch，1024，1】，【Batch，2048，1】
        self.conv1 =  nn.ModuleList()
        for i in input_channels:
            temp_part = nn.Sequential(

                nn.Conv1d(in_channels=in_channel, out_channels=i, kernel_size=1),
                nn.BatchNorm1d(i),
                nn.ReLU(inplace=True)

            )
            self.conv1.append(temp_part)

        self.conv2 = nn.ModuleList()
        for i in input_channels:
            temp_part_2 = nn.Sequential(
                nn.Conv1d(in_channels=i, out_channels=attention_channels, kernel_size=1),
                nn.BatchNorm1d(attention_channels),
                nn.ReLU(inplace=True)

            )
            self.conv2.append(temp_part_2)
        ## 通过attetnion 需要将他们对其到同一个尺度 return list[Batch,2048,1] * 4
        self.conv3 = nn.Conv1d(in_channels= attention_channels *2,out_channels=2, kernel_size=1)
        self.nonblock = NonLocalBlock(in_channels= attention_channels)
        self.conv4 = nn.ModuleList()
        for i in input_channels:
            temp_part_4 = nn.Sequential(
                nn.Conv1d(in_channels=attention_channels, out_channels=outchannels, kernel_size=1),
                nn.BatchNorm1d(outchannels),
                nn.ReLU(inplace=True)

            )
            self.conv4.append(temp_part_4)

        self.reg = nn.Sequential(
            nn.Linear(in_features=outchannels * len(self.input_channels) * 30, out_features= 2048 ),
            nn.Dropout(p=0.1),
            nn.ReLU(inplace=True),
            nn.Linear(2048,1)
        )

        # self.conv4 = nn.ModuleList([nn.Conv1d(in_channels= attention_channels,out_channels=outchannels,kernel_size=1) for i in range(len(input_channels))])

    def  forward(self,x):
        outs = [in_channel(x) for in_channel in self.conv1]
        outs = [in_channel(outs[idx])for idx,in_channel in enumerate(self.conv2)]

        if len(self.input_channels) == 4:
            conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
            conncat_tensor_01_conv = self.conv3(conncat_tensor_01)
            alpha_01 = F.softmax(conncat_tensor_01_conv,dim=1)
            alpha_01_1 = alpha_01[:,1,:].unsqueeze(dim=2)
            feature_attention_0 = self.nonblock(outs[0], outs[1], alpha_01_1, True)

            conncat_tensor_12 = torch.cat([outs[1], outs[2]], dim=1)
            conncat_tensor_12_conv = self.conv3(conncat_tensor_12)
            alpha_12 = F.softmax(conncat_tensor_12_conv, dim=1)
            alpha_12_2 = alpha_12[:, 1, :].unsqueeze(dim=2)
            feature_attention_1 = self.nonblock(outs[1], outs[2], alpha_12_2, True)

            conncat_tensor_23 = torch.cat([outs[2], outs[3]], dim=1)
            conncat_tensor_23_conv = self.conv3(conncat_tensor_23)
            alpha_23 = F.softmax(conncat_tensor_23_conv, dim=1)
            alpha_23_3 = alpha_23[:, 1, :].unsqueeze(dim=2)
            feature_attention_2 = self.nonblock(outs[2], outs[3], alpha_23_3, True)

            conncat_tensor_30 = torch.cat([outs[3], outs[0]], dim=1)
            conncat_tensor_30_conv = self.conv3(conncat_tensor_30)
            alpha_30 = F.softmax(conncat_tensor_30_conv, dim=1)
            alpha_30_3 = alpha_30[:, 1, :].unsqueeze(dim=2)
            feature_attention_3 = self.nonblock(outs[3], outs[0], alpha_30_3, True)

            outs = [feature_attention_0, feature_attention_1, feature_attention_2, feature_attention_3]
        elif len(self.input_channels) == 3:
            conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
            conncat_tensor_01_conv = self.conv3(conncat_tensor_01)
            alpha_01 = F.softmax(conncat_tensor_01_conv, dim=1)
            alpha_01_1 = alpha_01[:, 1, :].unsqueeze(dim=2)
            feature_attention_0 = self.nonblock(outs[0], outs[1], alpha_01_1, True)

            conncat_tensor_12 = torch.cat([outs[1], outs[2]], dim=1)
            conncat_tensor_12_conv = self.conv3(conncat_tensor_12)
            alpha_12 = F.softmax(conncat_tensor_12_conv, dim=1)
            alpha_12_2 = alpha_12[:, 1, :].unsqueeze(dim=2)
            feature_attention_1 = self.nonblock(outs[1], outs[2], alpha_12_2, True)

            conncat_tensor_20 = torch.cat([outs[2], outs[0]], dim=1)
            conncat_tensor_20_conv = self.conv2(conncat_tensor_20)
            alpha_20 = F.softmax(conncat_tensor_20_conv, dim=1)
            alpha_20_2 = alpha_20[:, 1, :].unsqueeze(dim=2)
            feature_attention_2 = self.nonblock(outs[2], outs[0], alpha_20_2, True)

            outs = [feature_attention_0, feature_attention_1, feature_attention_2]

        outs = [in_channel(outs[idx]) for idx, in_channel in enumerate(self.conv4)]

        input_feature = torch.cat(outs,dim = 1 )
        input_feature = input_feature.view(input_feature.shape[0],-1)
        outs = self.reg(input_feature)


        return outs


####　测试部分
## 现在我们得到了多个尺度的特征
#
# x1 = torch.rand([30,2048,1])
# x2 = torch.rand([30,2048,1])
# x3 = torch.rand([30,2048,1])
# x4 = torch.rand([30,2048,1])
# outs = [x1,x2,x3,x4]
# ### 1. concat feature
#
# conncat_tensor_01 = torch.cat([outs[0], outs[1]], dim=1)
# print(conncat_tensor_01.shape)
# conv1 = nn.Conv1d(in_channels=2048 *2 , out_channels=2, kernel_size=1)
# conncat_tensor_01_conv = conv1(conncat_tensor_01)
# alpha_01 = F.softmax(conncat_tensor_01_conv,dim=1)
# alpha_0 = alpha_01[:,0,:]
# alpha_1 = alpha_01[:,1,:].unsqueeze(dim=2)






# nonblock = NonLocalBlock(in_channels= 2048)
# temp_feature_0 = nonblock(outs[0], outs[1], alpha_1, False)



# conv1 = nn.Conv1d(in_channels=2048 * 2 ,out_channels=2,kernel_size=1)
# print(conv1(conncat_tensor_01).shape)

# feature = torch.rand([2,2048,30])
# in_channel = 2048
# input_channels = [256,512,1024,2048]
# attention_channels = 2048
# outchannels = 1024
# model = MTA(in_channel = in_channel, input_channels=input_channels,attention_channels= attention_channels,outchannels=outchannels)
# print(model)
#
# results = model(feature)
#
# print(results.shape)




#### AU
in_channel = 31
input_channels = [4, 8, 16, 31]
attention_channels = 8
outchannels = 16
model = MTA(in_channel=in_channel, input_channels=input_channels, attention_channels=attention_channels,
            outchannels=outchannels)

#### 读取权重

In [39]:
model.load_state_dict(torch.load("/hy-tmp/AU_pth/Epcoh_171_Rmse_6.135608673095703_PCC_0.38557139188424844_CCC_0.21280180728989406.pth"))

<All keys matched successfully>

### AU-NS

#### 加载模型

In [41]:
import torch
import torch.nn as nn
# 计算无关特征网络
class unrelated_conv(nn.Module):
    def __init__(self):
        super(unrelated_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 2048, out_channels= 512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512,out_channels=128,kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)

    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        rough_result= self.maxpool_2(temp2_relu)

        rough_result = self.conv3(rough_result)
        rough_result = self.relu_3(rough_result)
        rough_result = self.maxpool_3(rough_result)




        return rough_result

# 计算精细特征网络
class related_conv(nn.Module):
    def __init__(self):
        super(related_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2048, out_channels=512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512, out_channels=128, kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)



    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        smooth_result = self.maxpool_2(temp2_relu)

        smooth_result = self.conv3(smooth_result)
        smooth_result = self.relu_3(smooth_result)
        smooth_result = self.maxpool_3(smooth_result)


        return smooth_result
#输入数据为精细特征，预测结果
class predict_part(nn.Module):
    def __init__(self):
        super(predict_part, self).__init__()
        self.fc1 = nn.Linear(in_features=32,out_features=1)
        self.relu = nn.ReLU(True)
        self.fc2 = nn.Linear(in_features=64,out_features=1)
    def forward(self,x):
        x_in = x.reshape(x.shape[0],-1)
        x_in = self.relu(x_in)
        predict_temp1 = self.fc1(x_in)
        # predict_temp2 = self.relu(predict_temp1)
        # predict_result = self.relu(self.fc2(predict_temp2))

        return predict_temp1
#特征复原。。
class encoder_image(nn.Module):
    def __init__(self):
        super(encoder_image, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=128, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=512, kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=512, out_channels=2048, kernel_size=1)

    def forward(self, smooth_feature, rough_feature):
        encode_feature = torch.add(smooth_feature, rough_feature)
        encode_1 = self.conv1(encode_feature)
        x_hat = self.conv2(encode_1)

        x_hat = self.conv3(x_hat)
        return x_hat



class DSN(nn.Module):
    def __init__(self):
        super(DSN, self).__init__()
        self.unrelated_conv = unrelated_conv()
        self.related_conv = related_conv()
        self.predict_part = predict_part()
        self.encoder = encoder_image()

    def forward(self,X_feature):

        unrealted_data = self.unrelated_conv(X_feature)


        realted_data = self.related_conv(X_feature)


        predict_result  = self.predict_part(realted_data)


        encode_result = self.encoder(unrealted_data, realted_data)

        return unrealted_data, realted_data, predict_result, encode_result

# model = DSN()
# x = torch.rand([600,1,2048])
# x = x.permute(0,2,1)
# print(f'x:{x.shape}')
# unrealted_data, realted_data, predict_result, encode_result = model(x)
# print(f'unrealted_data:{unrealted_data.shape}')
# print(f'predict_Result :{predict_result.shape}')
# print(f'related_data:{realted_data.shape}')
# print(f'encode_result:{encode_result.shape}')
model = DSN()

#### 读取权重

In [42]:
model.load_state_dict(torch.load("/hy-tmp/AU_pth/Epcoh_4678_Rmse_6.085559368133545_PCC_0.41137230431761634_CCC_0.26710996359867084.pth"))

<All keys matched successfully>